In [2]:
import numpy as np
import cv2
import torch
import time
from torchvision import transforms
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts
from models.yolo import Model

# Initialize device
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Load YOLOv7-pose model
weights = torch.load('yolov7-w6-pose.pt', map_location=device, weights_only=False)
model = weights['model'].float().eval()
if torch.cuda.is_available():
    model.half().to(device)

# Parameters from journal
ALPHA = 0.5            # Height adjustment factor
SPEED_THRESHOLD = 100  # Speed threshold (pixels/second)
ANGLE_THRESHOLD = 45   # Angle threshold (degrees)
TARGET_FPS = 25        # Target processing FPS
KP_CONFIDENCE = 0.5    # Keypoint confidence threshold

# Tracking variables
prev_shoulder_y = None
prev_time = None
fall_counter = 0       # To prevent temporary false positives
FALL_FRAMES_THRESHOLD = 3  # Minimum consecutive frames to confirm fall

def calculate_length_factor(shoulder, hip):
    """Calculate L factor (Euclidean distance between shoulder and hip)"""
    return np.sqrt((shoulder[0] - hip[0])**2 + (shoulder[1] - hip[1])**2)

def calculate_angle(a, b, c):
    """Calculate angle between three points (in degrees)"""
    ba = np.array(a) - np.array(b)
    bc = np.array(c) - np.array(b)
    cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
    angle = np.arccos(np.clip(cosine_angle, -1, 1))
    return np.degrees(angle)

def detect_fall(keypoints, vertical_speed):
    """Implement fall detection algorithm from the journal"""
    global fall_counter
    
    # Keypoint indices (YOLOv7-pose format)
    LEFT_SHOULDER, RIGHT_SHOULDER = 5, 6
    LEFT_HIP, RIGHT_HIP = 11, 12
    LEFT_ANKLE, RIGHT_ANKLE = 15, 16
    
    # Extract keypoints with confidence check
    kpts = {
        'left_shoulder': keypoints[LEFT_SHOULDER*3:(LEFT_SHOULDER+1)*3],
        'right_shoulder': keypoints[RIGHT_SHOULDER*3:(RIGHT_SHOULDER+1)*3],
        'left_hip': keypoints[LEFT_HIP*3:(LEFT_HIP+1)*3],
        'right_hip': keypoints[RIGHT_HIP*3:(RIGHT_HIP+1)*3],
        'left_ankle': keypoints[LEFT_ANKLE*3:(LEFT_ANKLE+1)*3],
        'right_ankle': keypoints[RIGHT_ANKLE*3:(RIGHT_ANKLE+1)*3]
    }
    
    # Filter keypoints with low confidence
    if any(kp[2] < KP_CONFIDENCE for kp in kpts.values()):
        fall_counter = max(0, fall_counter - 1)
        return False
    
    # Calculate length factor (average of left and right)
    L_left = calculate_length_factor(kpts['left_shoulder'][:2], kpts['left_hip'][:2])
    L_right = calculate_length_factor(kpts['right_shoulder'][:2], kpts['right_hip'][:2])
    L_factor = (L_left + L_right) / 2
    
    # Criterion 1: Shoulder low relative to feet
    shoulder_low = (
        (kpts['left_shoulder'][1] <= kpts['left_ankle'][1] + ALPHA * L_factor) or
        (kpts['right_shoulder'][1] <= kpts['right_ankle'][1] + ALPHA * L_factor)
    )
    
    # Criterion 2: Hip low relative to feet
    hip_low = (
        (kpts['left_hip'][1] <= kpts['left_ankle'][1] + ALPHA * L_factor) or
        (kpts['right_hip'][1] <= kpts['right_ankle'][1] + ALPHA * L_factor)
    )
    
    # Criterion 3: Horizontal position (height < width)
    avg_shoulder_y = (kpts['left_shoulder'][1] + kpts['right_shoulder'][1]) / 2
    avg_ankle_y = (kpts['left_ankle'][1] + kpts['right_ankle'][1]) / 2
    body_height = abs(avg_shoulder_y - avg_ankle_y)
    body_width = abs(kpts['left_shoulder'][0] - kpts['right_shoulder'][0])
    horizontal_position = body_height < body_width
    
    # Criterion 4: Sharp angles or high speed
    left_angle = calculate_angle(
        kpts['left_shoulder'][:2],
        kpts['left_hip'][:2],
        kpts['left_ankle'][:2]
    )
    right_angle = calculate_angle(
        kpts['right_shoulder'][:2],
        kpts['right_hip'][:2],
        kpts['right_ankle'][:2]
    )
    min_angle = min(left_angle, right_angle)
    
    # Combine all criteria
    fall_conditions = (shoulder_low and hip_low and horizontal_position and 
                      (abs(vertical_speed) > SPEED_THRESHOLD or min_angle < ANGLE_THRESHOLD))
    
    # Require multiple consecutive frames to confirm fall
    if fall_conditions:
        fall_counter += 1
    else:
        fall_counter = max(0, fall_counter - 1)
    
    return fall_counter >= FALL_FRAMES_THRESHOLD

def process_video(video_path):
    """Process video stream for fall detection"""
    global prev_shoulder_y, prev_time, fall_counter
    
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print("Error opening video file")
        return
    
    # Frame rate adjustment
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    if original_fps <= 0:
        original_fps = 25  # Default if cannot read FPS
    skip_frames = max(1, int(round(original_fps / TARGET_FPS)))
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Skip frames to maintain target FPS
        if int(cap.get(cv2.CAP_PROP_POS_FRAMES)) % skip_frames != 0:
            continue
        
        # Preprocess frame
        image = letterbox(frame, 640, stride=64, auto=True)[0]
        image_tensor = transforms.ToTensor()(image).unsqueeze(0)
        
        if torch.cuda.is_available():
            image_tensor = image_tensor.half().to(device)
        
        # Run inference
        with torch.no_grad():
            output, _ = model(image_tensor)
            output = non_max_suppression_kpt(output, 0.25, 0.65, 
                                           nc=model.yaml['nc'], 
                                           nkpt=model.yaml['nkpt'], 
                                           kpt_label=True)
            output = output_to_keypoint(output)
        
        vertical_speed = 0
        current_time = time.perf_counter()
        fall_detected = False
        
        if output.shape[0] > 0:
            keypoints = output[0, 7:].T
            
            # Calculate vertical speed (shoulder movement)
            left_shoulder_y = keypoints[5*3+1]
            right_shoulder_y = keypoints[6*3+1]
            current_shoulder_y = (left_shoulder_y + right_shoulder_y) / 2
            
            if prev_shoulder_y is not None and prev_time is not None:
                time_diff = current_time - prev_time
                if time_diff > 0:
                    vertical_speed = (current_shoulder_y - prev_shoulder_y) / time_diff
            
            prev_shoulder_y = current_shoulder_y
            prev_time = current_time
            
            # Detect fall
            fall_detected = detect_fall(keypoints, vertical_speed)
            
            # Prepare visualization
            nimg = image_tensor[0].permute(1, 2, 0).cpu().numpy() * 255
            nimg = cv2.cvtColor(nimg.astype(np.uint8), cv2.COLOR_RGB2BGR)
            plot_skeleton_kpts(nimg, keypoints, 3)
            
            # Calculate angles for display
            left_angle = calculate_angle(
                keypoints[5*3:5*3+2],  # left shoulder
                keypoints[11*3:11*3+2], # left hip
                keypoints[15*3:15*3+2]  # left ankle
            )
            right_angle = calculate_angle(
                keypoints[6*3:6*3+2],  # right shoulder
                keypoints[12*3:12*3+2], # right hip
                keypoints[16*3:16*3+2]  # right ankle
            )
            min_angle = min(left_angle, right_angle)
            
            # Display information
            status_text = "Normal"
            color = (0, 255, 0)  # Green
            if fall_detected:
                status_text = "FALL DETECTED!"
                color = (0, 0, 255)  # Red
            
            cv2.putText(nimg, status_text, (50, 50), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
            cv2.putText(nimg, f"Speed: {vertical_speed:.1f} px/s", (50, 90), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
            cv2.putText(nimg, f"Angle: {min_angle:.1f}°", (50, 130), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
            
            # Show result
            cv2.imshow("Fall Detection", nimg)
        else:
            prev_shoulder_y = None
            prev_time = None
            fall_counter = 0
            cv2.imshow("Fall Detection", frame)
        
        if cv2.waitKey(1) == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    video_path = "C:\\Users\\LENOVO\\Documents\\A Skripsi\\datasets\\FallDataset\\Dataset\\Coffee_room_01\\Videos\\video (1).avi"
    process_video(video_path)

In [1]:
import numpy as np
import cv2
import torch
import time
from torchvision import transforms
from utils.datasets import letterbox
from utils.general import non_max_suppression_kpt
from utils.plots import output_to_keypoint, plot_skeleton_kpts
from models.yolo import Model

class FallDetector:
    def __init__(self):
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.load_model()
        self.initialize_parameters()
        self.reset_tracking_variables()
        
    def load_model(self):
        """Load YOLOv7-pose model with safety checks"""
        torch.serialization.add_safe_globals([Model])
        weights = torch.load('yolov7-w6-pose.pt', map_location=self.device, weights_only=False)
        self.model = weights['model'].float().eval()
        if torch.cuda.is_available():
            self.model.half().to(self.device)
    
    def initialize_parameters(self):
        """Set parameters according to the journal"""
        self.ALPHA = 0.5            # Height adjustment factor
        self.SPEED_THRESHOLD = 100   # Speed threshold (pixels/second)
        self.ANGLE_THRESHOLD = 45    # Angle threshold (degrees)
        self.TARGET_FPS = 25         # Target processing FPS
        self.KP_CONFIDENCE = 0.5    # Keypoint confidence threshold
        self.FALL_FRAMES_THRESHOLD = 3  # Minimum consecutive frames to confirm fall
    
    def reset_tracking_variables(self):
        """Reset variables for new detection sequence"""
        self.prev_shoulder_y = None
        self.prev_time = None
        self.fall_counter = 0
        self.frame_count = 0
    
    @staticmethod
    def calculate_length_factor(shoulder, hip):
        """Calculate L factor (Euclidean distance between shoulder and hip)"""
        return np.sqrt((shoulder[0] - hip[0])**2 + (shoulder[1] - hip[1])**2)
    
    @staticmethod
    def calculate_angle(a, b, c):
        """Calculate angle between three points (in degrees)"""
        ba = np.array(a) - np.array(b)
        bc = np.array(c) - np.array(b)
        cosine_angle = np.dot(ba, bc) / (np.linalg.norm(ba) * np.linalg.norm(bc))
        angle = np.arccos(np.clip(cosine_angle, -1, 1))
        return np.degrees(angle)
    
    def detect_fall(self, keypoints):
        """
        Implement complete fall detection algorithm from the journal
        with all four criteria and knee points integration
        """
        # Keypoint indices (YOLOv7-pose format)
        LEFT_SHOULDER, RIGHT_SHOULDER = 5, 6
        LEFT_HIP, RIGHT_HIP = 11, 12
        LEFT_KNEE, RIGHT_KNEE = 13, 14
        LEFT_ANKLE, RIGHT_ANKLE = 15, 16
        
        # Extract keypoints with confidence check
        kpts = {
            'left_shoulder': keypoints[LEFT_SHOULDER*3:(LEFT_SHOULDER+1)*3],
            'right_shoulder': keypoints[RIGHT_SHOULDER*3:(RIGHT_SHOULDER+1)*3],
            'left_hip': keypoints[LEFT_HIP*3:(LEFT_HIP+1)*3],
            'right_hip': keypoints[RIGHT_HIP*3:(RIGHT_HIP+1)*3],
            'left_knee': keypoints[LEFT_KNEE*3:(LEFT_KNEE+1)*3],
            'right_knee': keypoints[RIGHT_KNEE*3:(RIGHT_KNEE+1)*3],
            'left_ankle': keypoints[LEFT_ANKLE*3:(LEFT_ANKLE+1)*3],
            'right_ankle': keypoints[RIGHT_ANKLE*3:(RIGHT_ANKLE+1)*3]
        }
        
        # Filter keypoints with low confidence
        if any(kp[2] < self.KP_CONFIDENCE for kp in kpts.values()):
            self.fall_counter = max(0, self.fall_counter - 1)
            return False
        
        # Calculate vertical speed (shoulder movement)
        current_shoulder_y = (kpts['left_shoulder'][1] + kpts['right_shoulder'][1]) / 2
        vertical_speed = 0
        
        if self.prev_shoulder_y is not None and self.prev_time is not None:
            time_diff = time.perf_counter() - self.prev_time
            if time_diff > 0:
                vertical_speed = (current_shoulder_y - self.prev_shoulder_y) / time_diff
        
        self.prev_shoulder_y = current_shoulder_y
        self.prev_time = time.perf_counter()
        
        # Calculate length factor (average of left and right)
        L_left = self.calculate_length_factor(kpts['left_shoulder'][:2], kpts['left_hip'][:2])
        L_right = self.calculate_length_factor(kpts['right_shoulder'][:2], kpts['right_hip'][:2])
        L_factor = (L_left + L_right) / 2
        
        # Criterion 1: Shoulder low relative to feet (with length factor adjustment)
        shoulder_low = (
            (kpts['left_shoulder'][1] <= kpts['left_ankle'][1] + self.ALPHA * L_factor) or
            (kpts['right_shoulder'][1] <= kpts['right_ankle'][1] + self.ALPHA * L_factor)
        )
        
        # Criterion 2: Hip low relative to feet
        hip_low = (
            (kpts['left_hip'][1] <= kpts['left_ankle'][1] + self.ALPHA * L_factor) or
            (kpts['right_hip'][1] <= kpts['right_ankle'][1] + self.ALPHA * L_factor)
        )
        
        # Criterion 3: Horizontal position (height < width)
        avg_shoulder_y = (kpts['left_shoulder'][1] + kpts['right_shoulder'][1]) / 2
        avg_ankle_y = (kpts['left_ankle'][1] + kpts['right_ankle'][1]) / 2
        body_height = abs(avg_shoulder_y - avg_ankle_y)
        body_width = abs(kpts['left_shoulder'][0] - kpts['right_shoulder'][0])
        horizontal_position = body_height < body_width
        
        # Criterion 4: Sharp angles (using knee points) or high speed
        left_leg_angle = self.calculate_angle(
            kpts['left_hip'][:2],
            kpts['left_knee'][:2],
            kpts['left_ankle'][:2]
        )
        right_leg_angle = self.calculate_angle(
            kpts['right_hip'][:2],
            kpts['right_knee'][:2],
            kpts['right_ankle'][:2]
        )
        min_angle = min(left_leg_angle, right_leg_angle)
        
        # Combine all criteria exactly as in the journal
        fall_conditions = (
            shoulder_low and 
            hip_low and 
            horizontal_position and
            (abs(vertical_speed) > self.SPEED_THRESHOLD or min_angle < self.ANGLE_THRESHOLD)
        )
        
        # Require multiple consecutive frames to confirm fall
        if fall_conditions:
            self.fall_counter += 1
        else:
            self.fall_counter = max(0, self.fall_counter - 1)
        
        return self.fall_counter >= self.FALL_FRAMES_THRESHOLD
    
    def process_frame(self, frame):
        """Process a single frame for fall detection"""
        # Preprocess frame
        image = letterbox(frame, 640, stride=64, auto=True)[0]
        image_tensor = transforms.ToTensor()(image).unsqueeze(0)
        
        if torch.cuda.is_available():
            image_tensor = image_tensor.half().to(self.device)
        
        # Run inference
        with torch.no_grad():
            output, _ = self.model(image_tensor)
            output = non_max_suppression_kpt(
                output, 0.25, 0.65, 
                nc=self.model.yaml['nc'], 
                nkpt=self.model.yaml['nkpt'], 
                kpt_label=True
            )
            output = output_to_keypoint(output)
        
        fall_detected = False
        visualization_img = frame.copy()
        
        if output.shape[0] > 0:
            keypoints = output[0, 7:].T
            plot_skeleton_kpts(visualization_img, keypoints, 3)
            
            # Detect fall with all journal criteria
            fall_detected = self.detect_fall(keypoints)
            
            # Calculate angles for display
            left_angle = self.calculate_angle(
                keypoints[11*3:11*3+2],  # left hip
                keypoints[13*3:13*3+2],   # left knee
                keypoints[15*3:15*3+2]    # left ankle
            )
            right_angle = self.calculate_angle(
                keypoints[12*3:12*3+2],   # right hip
                keypoints[14*3:14*3+2],   # right knee
                keypoints[16*3:16*3+2]    # right ankle
            )
            min_angle = min(left_angle, right_angle)
            
            # Display information
            status_text = "Normal"
            color = (0, 255, 0)  # Green
            if fall_detected:
                status_text = "FALL DETECTED!"
                color = (0, 0, 255)  # Red
            
            cv2.putText(visualization_img, status_text, (50, 50), 
                       cv2.FONT_HERSHEY_SIMPLEX, 1, color, 2)
            cv2.putText(visualization_img, f"Min Angle: {min_angle:.1f}°", (50, 90), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 0), 2)
        
        return visualization_img, fall_detected

def main():
    detector = FallDetector()
    
    video_path = "C:\\Users\\LENOVO\\Documents\\A Skripsi\\datasets\\FallDataset\\Dataset\\Coffee_room_01\\Videos\\video (1).avi"
    cap = cv2.VideoCapture(video_path)
    
    if not cap.isOpened():
        print("Error opening video file")
        return
    
    # Frame rate adjustment
    original_fps = cap.get(cv2.CAP_PROP_FPS)
    if original_fps <= 0:
        original_fps = 25  # Default if cannot read FPS
    skip_frames = max(1, int(round(original_fps / detector.TARGET_FPS)))
    
    print(f"Processing video at target FPS: {detector.TARGET_FPS} (skipping {skip_frames-1} frames)")
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        
        # Skip frames to maintain target FPS
        if int(cap.get(cv2.CAP_PROP_POS_FRAMES)) % skip_frames != 0:
            continue
        
        # Process frame
        processed_frame, fall_detected = detector.process_frame(frame)
        
        # Display result
        cv2.imshow("Fall Detection", processed_frame)
        
        if cv2.waitKey(1) == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()

Processing video at target FPS: 25 (skipping 0 frames)


C:\Users\LENOVO\anaconda3\envs\yolov7_env\lib\site-packages\torch\functional.py:539: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\native\TensorShape.cpp:3638.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
